In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,scale
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss

In [2]:
# Dates have to extracted
# data = pd.read_csv('train.csv', parse_dates=['Dates'])
data = pd.read_csv('./dataset/train.csv', parse_dates=['Dates'])

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 867873 entries, 0 to 867872
Data columns (total 10 columns):
Dates         867873 non-null datetime64[ns]
Category      867873 non-null object
Descript      867873 non-null object
DayOfWeek     867873 non-null object
PdDistrict    867873 non-null object
Resolution    867873 non-null object
Address       867873 non-null object
X             867873 non-null float64
Y             867873 non-null float64
Id            867873 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 66.2+ MB


In [4]:
# No null values in the data-frame
data.isnull().values.any()

False

In [5]:
# Dates

data_week_dict = {
    'Monday': 1,
    'Tuesday':2,
    'Wednesday':3,
    'Thursday':4,
    'Friday':5,
    'Saturday':6,
    'Sunday':7
}

data['Hour'] = data.Dates.dt.hour
data['Minutes'] = data.Dates.dt.minute
data['Year'] = data.Dates.dt.year
data['Month'] = data.Dates.dt.month
data['Day'] = data.Dates.dt.day
data['DayOfWeekNum'] = data['DayOfWeek'].replace(data_week_dict)

In [6]:
data.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,Hour,Minutes,Year,Month,Day,DayOfWeekNum
0,2013-06-28 17:40:00,SEX OFFENSES FORCIBLE,"FORCIBLE RAPE, BODILY FORCE",Friday,MISSION,NONE,2100 Block of MISSION ST,-122.419331,37.762264,141546,17,40,2013,6,28,5
1,2004-02-19 02:46:00,LIQUOR LAWS,CONSUMING ALCOHOL IN PUBLIC VIEW,Thursday,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411340,37.781271,794152,2,46,2004,2,19,4
2,2007-11-14 00:01:00,FRAUD,"CREDIT CARD, THEFT BY USE OF",Wednesday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,531205,0,1,2007,11,14,3
3,2007-12-27 18:30:00,ROBBERY,ROBBERY OF A CHAIN STORE WITH A GUN,Thursday,BAYVIEW,DISTRICT ATTORNEY REFUSES TO PROSECUTE,2400 Block of SAN BRUNO AV,-122.404715,37.730161,523137,18,30,2007,12,27,4
4,2012-09-09 17:02:00,OTHER OFFENSES,PROBATION VIOLATION,Sunday,SOUTHERN,"ARREST, BOOKED",4TH ST / STEVENSON ST,-122.405239,37.785265,200968,17,2,2012,9,9,7


In [7]:
labelencoder = LabelEncoder()

In [8]:
# Address
# Think of an alternative solution - Issue when the attribute contains two streets
# Logically, 'street A / street B' == 'street B / street A'
# Such cases would fails this step

def street_from_address(address):
    street = address.split()
    return (' '.join(street[-2:]))

data['Street'] = data['Address'].apply(lambda a:street_from_address(a))


In [9]:
# streets = data['Street'].unique()
# data_street_dict = dict()
# ctr = 1
# for s in streets:
#     data_street_dict[s] = ctr
#     ctr += 1

# data['StreetNum'] = data['Street'].replace(data_street_dict)
data['StreetNum'] = labelencoder.fit_transform(data['Street'])

In [10]:
# Police Department District

# district = data['PdDistrict'].unique()
# data_district_dict = {}
# ctr = 1

# for d in district:
#     data_district_dict[d] = ctr
#     ctr += 1

# data['PdDistrictNum'] = data['PdDistrict'].replace(data_district_dict)
data['PdDistrictNum'] = labelencoder.fit_transform(data['PdDistrict'])

In [11]:
# X and Y

data = data[data.X < -121]
data = data[data.Y < 40]

In [12]:
# Resolution

# resolutions = data['Resolution'].unique()
# data_resolution_dict = dict()
# ctr = 1
# for r in resolutions:
#     data_resolution_dict[r] = ctr
#     ctr += 1

# data['ResolutionNum'] = data['Resolution'].replace(data_resolution_dict)
data['ResolutionNum'] = labelencoder.fit_transform(data['Resolution'])

In [13]:
# Target Variable - Category

# target = data['Category'].unique()
# data_category_dict = dict()
# ctr = 1
# for t in target:
#     data_category_dict[t] = ctr
#     ctr += 1

# data['CategoryNum'] = data['Category'].replace(data_category_dict)
data['CategoryNum'] = labelencoder.fit_transform(data['Category'])

In [14]:
def getCapsAddress(i):
    s=''
    for j in i.split():
        if(j.isupper()):
            s=s+' '+j
    return s[1:]
data['newAddress'] = data.Address.apply(lambda a:getCapsAddress(a))
data

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Id,...,Year,Month,Day,DayOfWeekNum,Street,StreetNum,PdDistrictNum,ResolutionNum,CategoryNum,newAddress
0,2013-06-28 17:40:00,SEX OFFENSES FORCIBLE,"FORCIBLE RAPE, BODILY FORCE",Friday,MISSION,NONE,2100 Block of MISSION ST,-122.419331,37.762264,141546,...,2013,6,28,5,MISSION ST,1272,3,11,27,MISSION ST
1,2004-02-19 02:46:00,LIQUOR LAWS,CONSUMING ALCOHOL IN PUBLIC VIEW,Thursday,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411340,37.781271,794152,...,2004,2,19,4,MARKET ST,1200,7,0,17,MARKET ST
2,2007-11-14 00:01:00,FRAUD,"CREDIT CARD, THEFT BY USE OF",Wednesday,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,531205,...,2007,11,14,3,BRYANT ST,314,7,11,13,BRYANT ST
3,2007-12-27 18:30:00,ROBBERY,ROBBERY OF A CHAIN STORE WITH A GUN,Thursday,BAYVIEW,DISTRICT ATTORNEY REFUSES TO PROSECUTE,2400 Block of SAN BRUNO AV,-122.404715,37.730161,523137,...,2007,12,27,4,BRUNO AV,310,0,4,24,SAN BRUNO AV
4,2012-09-09 17:02:00,OTHER OFFENSES,PROBATION VIOLATION,Sunday,SOUTHERN,"ARREST, BOOKED",4TH ST / STEVENSON ST,-122.405239,37.785265,200968,...,2012,9,9,7,STEVENSON ST,1776,7,0,21,4TH ST STEVENSON ST
5,2014-05-14 06:14:00,WARRANTS,ENROUTE TO DEPARTMENT OF CORRECTIONS,Wednesday,MISSION,"ARREST, BOOKED",15TH ST / NATOMA ST,-122.418405,37.766760,75749,...,2014,5,14,3,NATOMA ST,1329,3,0,34,15TH ST NATOMA ST
6,2006-08-09 23:45:00,BURGLARY,"BURGLARY, UNLAWFUL ENTRY",Wednesday,CENTRAL,"ARREST, BOOKED",0 Block of MARK LN,-122.404651,37.790442,617521,...,2006,8,9,3,MARK LN,1199,1,0,4,MARK LN
7,2012-11-16 12:41:00,SUSPICIOUS OCC,SUSPICIOUS OCCURRENCE,Friday,INGLESIDE,NONE,100 Block of PUTNAM ST,-122.410871,37.737526,186953,...,2012,11,16,5,PUTNAM ST,1517,2,11,30,PUTNAM ST
8,2007-02-05 23:53:00,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Monday,BAYVIEW,"ARREST, CITED",HALE ST / SAN BRUNO AV,-122.405911,37.732964,584559,...,2007,2,5,1,BRUNO AV,310,0,1,21,HALE ST SAN BRUNO AV
9,2006-02-07 07:41:00,DRUG/NARCOTIC,UNDER THE INFLUENCE OF CONTROLLED SUBSTANCES,Tuesday,NORTHERN,"ARREST, BOOKED",ELLIS ST / LARKIN ST,-122.417710,37.784236,653619,...,2006,2,7,2,LARKIN ST,1079,4,0,7,ELLIS ST LARKIN ST


In [15]:
le=LabelEncoder()
data['newAddressNum'] = le.fit_transform(data.newAddress)

In [16]:
corr = data.corr()
print(corr['CategoryNum'].sort_values(ascending=False))

CategoryNum      1.000000
ResolutionNum    0.039676
Hour             0.023701
Id               0.016881
DayOfWeekNum     0.000537
Day              0.000354
Month           -0.000360
StreetNum       -0.014526
Year            -0.016795
newAddressNum   -0.019639
Minutes         -0.021945
X               -0.030525
PdDistrictNum   -0.039477
Y               -0.058963
Name: CategoryNum, dtype: float64


In [17]:
data.drop('Dates', axis=1, inplace=True)
data.drop('Descript', axis=1, inplace=True)
data.drop('DayOfWeek', axis=1, inplace=True)
data.drop('PdDistrict', axis=1, inplace=True)
data.drop('Resolution', axis=1, inplace=True)
data.drop('Address', axis=1, inplace=True)
data.drop('Street', axis=1, inplace=True)
data.drop('Category', axis=1, inplace=True)
data.drop('Minutes', axis=1, inplace=True)
data.drop('newAddress', axis=1, inplace=True)
data.drop('Id',axis=1,inplace=True)

In [18]:
corr = data.corr()
print(corr['CategoryNum'].sort_values(ascending=False))

CategoryNum      1.000000
ResolutionNum    0.039676
Hour             0.023701
DayOfWeekNum     0.000537
Day              0.000354
Month           -0.000360
StreetNum       -0.014526
Year            -0.016795
newAddressNum   -0.019639
X               -0.030525
PdDistrictNum   -0.039477
Y               -0.058963
Name: CategoryNum, dtype: float64


In [19]:
# for i in data.CategoryNum.unique():
#     print(i,labelencoder.inverse_transform(data.CategoryNum.unique())[i])
#     data[data.CategoryNum==i].hist(bins=50, figsize=(20,15))
#     plt.show()
data.hist(bins=50, figsize=(20,15))
plt.show()

KeyboardInterrupt: 

In [ ]:
data.DayOfWeekNum.apply(np.log).hist()
plt.show()

In [ ]:
data.ResolutionNum.hist()
plt.show()

In [20]:
# Random seed has been set - As per the guidlines of the competition
train, test = train_test_split(data, test_size=0.3, random_state=3, shuffle=True)

In [21]:
y_train = train['CategoryNum']
X_train = train.drop(['CategoryNum'],axis=1)
y_test = test['CategoryNum']
X_test = test.drop(['CategoryNum'],axis=1)

In [46]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial',max_iter=1000)
clf.fit(X_train,y_train)
pred = clf.predict_proba(X_test)
log_loss(y_test,pred)

/Users/vibhavagarwal/miniconda2/envs/deeplearning/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


2.5023304997970617

In [45]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=14)
clf.fit(X_train,y_train)
# clf.predict(X_test),y_test
pred = clf.predict_proba(X_test)
log_loss(y_test,pred)

3.2087772316326237

In [47]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train,y_train)
pred = clf.predict_proba(X_test)
log_loss(y_test,pred)

2.492407041353744

In [ ]:
import xgboost as xgb

seed = 42

model = xgb.XGBClassifier(objective='multi:softprob', seed=seed)

In [ ]:
score = -1 * cross_val_score(model, X_train, y_train, scoring='neg_log_loss', cv=5).mean()

In [ ]:
print("Score = {0:.5f}".format(score))

TODO
- Pick relevant/independent columns and pass to the classifier
- Normalize the data
- Perform Exploratory Data Analysis, Visualization
- Finalize the feature list
- Build the models
- Voila!